In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/clicker/clicker_simple.ipynb)

# Clicker — Quick Demo

Pick atomic column positions on a HAADF-STEM image with a hexagonal lattice.
Click on bright atom columns to select up to 5 positions.

In [2]:
import numpy as np
from quantem.widget import Clicker


def make_haadf_stem(size=256, spacing=18, sigma=2.8):
    """HAADF-STEM image with atomic columns on a hexagonal lattice."""
    y, x = np.mgrid[:size, :size]
    img = np.random.normal(0.08, 0.015, (size, size))
    a1 = np.array([spacing, 0.0])
    a2 = np.array([spacing * 0.5, spacing * np.sqrt(3) / 2])
    for i in range(-1, size // spacing + 2):
        for j in range(-1, size // spacing + 2):
            cx = i * a1[0] + j * a2[0]
            cy = i * a1[1] + j * a2[1]
            if -spacing < cx < size + spacing and -spacing < cy < size + spacing:
                # Slight intensity variation (like mixed Z columns)
                intensity = 0.7 + 0.3 * ((i + j) % 3 == 0)
                img += intensity * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    # Add scan noise (horizontal streaks)
    scan_noise = np.random.normal(0, 0.01, (size, 1)) * np.ones((1, size))
    img += scan_noise
    return np.clip(img, 0, None).astype(np.float32)


haadf = make_haadf_stem()
w = Clicker(haadf, scale=1.0, max_points=5)
w

Clicker(256×256, pts=0)

## Inspect widget state

After clicking on atom columns above, run the cell below to see a full summary
of the widget state — image info, placed points with coordinates, display settings.

In [3]:
w.summary()

Clicker
════════════════════════════════
Image:    256×256
Data:     min=0.01407  max=1.11  mean=0.1898  dtype=float32
Display:  gray | auto contrast | linear
Points:   0/5
Marker:   circle red  size=12px


## Gallery Mode

Pass multiple images to pick points on each independently.
Click an image to select it, then click to place points.

In [4]:
# Create 3 HAADF images with different lattice spacings
imgs = [
    make_haadf_stem(size=128, spacing=14),
    make_haadf_stem(size=128, spacing=18),
    make_haadf_stem(size=128, spacing=22),
]
w_gallery = Clicker(imgs, ncols=3, max_points=3, labels=["d=14px", "d=18px", "d=22px"])
w_gallery

Clicker(3×128×128, pts=0)

In [5]:
w_gallery.summary()

Clicker
════════════════════════════════
Image:    3×128×128 (3 cols)
Data:     min=0.013  max=1.125  mean=0.2163  dtype=float32
Display:  gray | auto contrast | linear
Points [d=14px]: 0/3
Points [d=18px]: 0/3
Points [d=22px]: 0/3
Marker:   circle red  size=12px
